In [2]:
import pandas as pd
import numpy as np
import scipy.io
import mat73

from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVC

### Data loading and preprocessing

In [3]:
import os

DATA_DIR = ''
if 'google.colab' not in str(get_ipython()) and "anuja" in os.environ.get('USER'):
    DATA_DIR = 'data/'
    

In [4]:

#here we use these columns nly to get the subject ID
foof = pd.read_csv(DATA_DIR+"foof2features.csv")
foof = foof.rename(columns={"C1": "IDs" ,"C2": "Intercept", "C3": "Slope"})
foof

,IDs,Intercept,Slope
0,NDARAA075AMK,0.986272,1.825774
1,NDARAA112DMH,1.486650,1.888544
2,NDARAA117NEJ,1.593155,2.095749
3,NDARAA947ZG5,0.703331,1.724831
4,NDARAA948VFH,0.918020,1.749441
...,...,...,...
2037,NDARZN277NR6,1.351549,1.996940
2038,NDARZN578YDP,1.380795,2.036327
2039,NDARZN610GTY,0.339229,1.050644
2040,NDARZN677EYE,0.781225,1.470061


In [5]:
data = mat73.loadmat(DATA_DIR+'x_source.mat')  

# flattening
df2 = pd.DataFrame(data['x'].reshape((data['x'].shape[0], -1)))
df2 = np.array(df2)
df2 = df2.reshape((2041,68,391)) 
print(df2.shape)

(2041, 68, 391)


In [6]:
sparsed= np.concatenate([np.expand_dims(df2[:,:,i:i+10].mean(axis = 2), axis = 2) for i in range(0,381,10)], axis = 2)

#sanity check
df2[0,0,10:20].mean()
sparsed[0,0,1]
sparsed.shape


(2041, 68, 39)

In [7]:
#flattening and adding id
df2 = pd.DataFrame(sparsed.reshape((sparsed.shape[0], -1)))
df2.shape

(2041, 2652)

In [8]:
df2['IDs'] = foof['IDs']
df2.shape

(2041, 2653)

In [9]:
beh = pd.read_csv(DATA_DIR+"behaviorals.csv")
print('Before:'+str(beh.shape))

most_common_disorders = ['Attention-Deficit/Hyperactivity Disorder', 'Anxiety Disorders', 'Specific Learning Disorder',
                         'Autism Spectrum Disorder', 'Disruptive', 'No Diagnosis Given', 'Communication Disorder',
                         'Depressive Disorders']

# most_common_disorders = ['Other Neurodevelopmental Disorders', 'ADHD-Inattentive Type', 'ADHD-Combined Type', 'Anxiety Disorders', 'No Diagnosis Given', 'Depressive Disorders']

category_columns = ['DX_' + str(i).zfill(2) + '_Cat' for i in range(1, 11)] +\
                   ['DX_' + str(i).zfill(2) + '_Sub' for i in range(1, 11)]

# find users that have no diagnosis within these top diseases
# filtering should cahnge anything as this should also happen at a later stage
mask = None
for col in category_columns:
    mask_col = beh[col].isin(most_common_disorders)
    if mask is None:
        mask = mask_col
    else:
        mask = mask | mask_col

initial_size = beh.shape[0]
beh = beh[mask]
beh = beh.reset_index(drop=True)
new_size = beh.shape[0]
print('After:'+str(beh.shape))
print('Removing', initial_size - new_size,
      'patients as their diagnoses were very uncommon.')

Before:(3076, 177)
After:(2813, 177)
Removing 263 patients as their diagnoses were very uncommon.


In [10]:
no_diagnosis_given = 'No Diagnosis Given'

if no_diagnosis_given in most_common_disorders:
    no_diag_index = most_common_disorders.index(no_diagnosis_given)
    most_common_disorders = most_common_disorders[:no_diag_index] + \
        most_common_disorders[no_diag_index + 1:]

diagnoses_to_ids = {disorder: i for i, disorder in enumerate(most_common_disorders)}
diagnoses_to_ids

{'Attention-Deficit/Hyperactivity Disorder': 0,
 'Anxiety Disorders': 1,
 'Specific Learning Disorder': 2,
 'Autism Spectrum Disorder': 3,
 'Disruptive': 4,
 'Communication Disorder': 5,
 'Depressive Disorders': 6}

In [11]:
def get_disorder(data, row, index):
    disorder = data.iloc[row][category_columns[index]]

    if disorder == 'Neurodevelopmental Disorders':
        disorder = data.iloc[row][category_columns[index + 10]]

    return disorder

order_of_disorders = []
for k in range(beh.shape[0]):
    i = 0
    disorder = get_disorder(beh, k, i)
    disorders_patient = []
    while disorder != no_diagnosis_given and not pd.isnull(disorder):
        if disorder in diagnoses_to_ids:
            if diagnoses_to_ids[disorder] not in disorders_patient:
                disorders_patient.append(diagnoses_to_ids[disorder])
        i += 1
        if i == len(category_columns):
            break
        disorder = get_disorder(beh, k, i)

    order_of_disorders.append(disorders_patient)


In [12]:


max_len_order = np.max([len(x) for x in order_of_disorders])

# pad with a new token denoting the pad token
pad_token = len(most_common_disorders)
bod_token = len(most_common_disorders) + 1
eod_token = len(most_common_disorders) + 2

order_of_disorders = [[bod_token] + x + [eod_token] + [pad_token] * (max_len_order - len(x)) for x in order_of_disorders]

order_of_disorders = np.array(order_of_disorders)

classes = np.zeros((len(most_common_disorders),
                    beh.shape[0]), dtype=np.int32)

df_disorders = beh[category_columns]

for i, disorder in enumerate(most_common_disorders):
    mask = df_disorders.select_dtypes(include=[object]). \
        applymap(lambda x: disorder in x if pd.notnull(x) else False)

    disorder_df = df_disorders[mask.any(axis=1)]

    np.add.at(classes[i], disorder_df.index.values, 1)

behaviour_data_columns = beh.columns.values.astype(np.str)

columns_to_drop = behaviour_data_columns[
    np.flatnonzero(np.core.defchararray.find(behaviour_data_columns, 'DX') != -1)]

behaviour_data = beh.drop(columns=columns_to_drop)

for disorder, classification in zip(most_common_disorders, classes):
    behaviour_data[disorder] = classification

behaviour_data['order_diagnoses'] = list(order_of_disorders)


In [13]:
labels=behaviour_data[["IDs"]+list(most_common_disorders)]
labels

,IDs,Attention-Deficit/Hyperactivity Disorder,Anxiety Disorders,Specific Learning Disorder,Autism Spectrum Disorder,Disruptive,Communication Disorder,Depressive Disorders
0,NDARAA075AMK,0,0,0,0,0,0,0
1,NDARAA112DMH,1,0,0,0,1,0,0
2,NDARAA117NEJ,1,0,0,0,1,0,0
3,NDARAA306NT2,1,1,1,0,0,1,0
4,NDARAA504CRN,1,1,1,0,0,0,0
...,...,...,...,...,...,...,...,...
2808,NDARZZ007YMP,0,0,0,1,0,0,0
2809,NDARZZ740MLM,1,0,0,0,0,0,0
2810,NDARZZ810LVF,0,0,0,1,0,1,0
2811,NDARZZ830JM7,0,0,0,1,0,0,0


In [14]:

df = pd.merge(df2, labels, on='IDs', how='inner')
df.shape



(1835, 2660)

In [15]:

#removing NaNs
df = df.dropna()
df.shape

(1835, 2660)

### Data Split

In [16]:
x = df[df.columns.difference(['IDs']+most_common_disorders)]
y = df[most_common_disorders]

# summarize dataset shape
print(x.shape, y.shape)

(1835, 2652) (1835, 7)


In [17]:
#scaling features

# data normalization with sklearn
from sklearn.preprocessing import MinMaxScaler

# fit scaler on training data
norm = MinMaxScaler().fit(x)

# transform training data
x_norm = norm.transform(x)
x_norm = x


In [18]:


train_features, test_features, train_labels, test_labels = train_test_split(x, y, test_size=0.3, shuffle=True)

### Metrics

In [19]:
from sklearn.metrics import hamming_loss, accuracy_score
import sklearn.metrics as skm
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import numpy as np


def evaluate(y_test, y_pred):
    print("Accuracy:", accuracy_score(y_test, y_pred))
    print("Hamming Loss:", hamming_loss(y_test, y_pred))
    print("Classification Report:\n", skm.classification_report(y_test,y_pred, zero_division=1))
    print("Confusion matrix:\n", skm.multilabel_confusion_matrix(y_test, y_pred))

## Models

### Multi Output Classifier

In [20]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC as svm
from sklearn.multioutput import MultiOutputClassifier

forest = RandomForestClassifier(random_state=1)
lg = LogisticRegression()
svm = svm()
models = [lg, forest, svm]

for model in models:

    multi_output_model = MultiOutputClassifier(model, n_jobs=-1)
    multi_output_model.fit(train_features, train_labels)
    predicted_labels = multi_output_model.predict(test_features)
    print(str(model)+':')
    evaluate(test_labels, predicted_labels)

/home/ubuntu/anaconda3/envs/ai4halth/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/ubuntu/anaconda3/envs/ai4halth/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/lin

LogisticRegression():
Accuracy: 0.09437386569872959
Hamming Loss: 0.2870106300233342
Classification Report:
               precision    recall  f1-score   support

           0       0.64      0.66      0.65       357
           1       0.30      0.29      0.30       177
           2       0.25      0.15      0.19       142
           3       0.20      0.10      0.14        98
           4       0.17      0.07      0.10        82
           5       0.09      0.03      0.05        88
           6       0.04      0.02      0.03        51

   micro avg       0.43      0.33      0.37       995
   macro avg       0.24      0.19      0.21       995
weighted avg       0.36      0.33      0.34       995
 samples avg       0.54      0.43      0.34       995

Confusion matrix:
 [[[ 61 133]
  [120 237]]

 [[252 122]
  [125  52]]

 [[345  64]
  [121  21]]

 [[414  39]
  [ 88  10]]

 [[439  30]
  [ 76   6]]

 [[431  32]
  [ 85   3]]

 [[478  22]
  [ 50   1]]]
RandomForestClassifier(random_state=1):

### MLP

In [21]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

def get_mlp(n_inputs, n_outputs):
    model = Sequential()
    model.add(Dense(20, input_dim=n_inputs, kernel_initializer='he_uniform', activation='relu'))
    model.add(Dense(n_outputs, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam')
    return model

n_inputs, n_outputs = x.shape[1], y.shape[1]
mlp = get_mlp(n_inputs, n_outputs)
mlp.fit(train_features, train_labels, verbose=1, epochs=100)

2021-11-20 13:03:40.192619: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2021-11-20 13:03:40.221640: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-20 13:03:40.222306: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1561] Found device 0 with properties: 
pciBusID: 0000:00:06.0 name: Tesla T4 computeCapability: 7.5
coreClock: 1.59GHz coreCount: 40 deviceMemorySize: 14.75GiB deviceMemoryBandwidth: 298.08GiB/s
2021-11-20 13:03:40.222512: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2021-11-20 13:03:40.223702: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10
2021-11-20 13:03:40.225035: I tensorflow/stream_executor/platform/default/d

Epoch 1/100
 1/41 [..............................] - ETA: 0s - loss: 0.6083

2021-11-20 13:03:41.243512: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10


41/41 [==============================] - 0s 1ms/step - loss: 0.5163
Epoch 2/100
41/41 [==============================] - 0s 1ms/step - loss: 0.5038
Epoch 3/100
41/41 [==============================] - 0s 1ms/step - loss: 0.4933
Epoch 4/100
41/41 [==============================] - 0s 1ms/step - loss: 0.4938
Epoch 5/100
41/41 [==============================] - 0s 1ms/step - loss: 0.4920
Epoch 6/100
41/41 [==============================] - 0s 1ms/step - loss: 0.4815
Epoch 7/100
41/41 [==============================] - 0s 1ms/step - loss: 0.4792
Epoch 8/100
41/41 [==============================] - 0s 1ms/step - loss: 0.4821
Epoch 9/100
41/41 [==============================] - 0s 1ms/step - loss: 0.4742
Epoch 10/100
41/41 [==============================] - 0s 1ms/step - loss: 0.4681
Epoch 11/100
41/41 [==============================] - 0s 1ms/step - loss: 0.4694
Epoch 12/100
41/41 [==============================] - 0s 1ms/step - loss: 0.4629
Epoch 13/100
41/41 [=============================

In [22]:
predicted_labels_mlp = mlp.predict(test_features)
evaluate(test_labels, predicted_labels_mlp.round())

Accuracy: 0.11796733212341198
Hamming Loss: 0.27223230490018147
Classification Report:
               precision    recall  f1-score   support

           0       0.64      0.45      0.53       357
           1       0.31      0.32      0.32       177
           2       0.30      0.11      0.16       142
           3       0.50      0.03      0.06        98
           4       0.24      0.05      0.08        82
           5       0.11      0.01      0.02        88
           6       0.08      0.04      0.05        51

   micro avg       0.45      0.24      0.32       995
   macro avg       0.31      0.14      0.17       995
weighted avg       0.41      0.24      0.29       995
 samples avg       0.64      0.34      0.29       995

Confusion matrix:
 [[[103  91]
  [197 160]]

 [[252 122]
  [121  56]]

 [[372  37]
  [126  16]]

 [[450   3]
  [ 95   3]]

 [[456  13]
  [ 78   4]]

 [[455   8]
  [ 87   1]]

 [[477  23]
  [ 49   2]]]


### Binary Relevance
ignores the possible correlations between class labels

In [23]:
from skmultilearn.problem_transform import BinaryRelevance
from sklearn.naive_bayes import GaussianNB

classifier = BinaryRelevance(GaussianNB())
classifier.fit(test_features, test_labels)

BinaryRelevance(classifier=GaussianNB(), require_dense=[True, True])

In [24]:
predicted_labels_br = classifier.predict(test_features)
evaluate(test_labels, predicted_labels_br)

Accuracy: 0.16152450090744103
Hamming Loss: 0.23463831993777548
Classification Report:
               precision    recall  f1-score   support

           0       0.91      0.83      0.87       357
           1       0.63      0.82      0.72       177
           2       0.44      0.72      0.54       142
           3       0.44      0.66      0.53        98
           4       0.28      0.77      0.41        82
           5       0.38      0.73      0.50        88
           6       0.32      0.86      0.46        51

   micro avg       0.53      0.78      0.63       995
   macro avg       0.48      0.77      0.58       995
weighted avg       0.62      0.78      0.67       995
 samples avg       0.54      0.82      0.59       995

Confusion matrix:
 [[[165  29]
  [ 61 296]]

 [[289  85]
  [ 31 146]]

 [[277 132]
  [ 40 102]]

 [[369  84]
  [ 33  65]]

 [[308 161]
  [ 19  63]]

 [[359 104]
  [ 24  64]]

 [[405  95]
  [  7  44]]]


### Classfier Chains

In [25]:
from skmultilearn.problem_transform import ClassifierChain
from sklearn.linear_model import LogisticRegression

classifier = ClassifierChain(LogisticRegression())
classifier.fit(train_features, train_labels)
# we should optimise this a little

/home/ubuntu/anaconda3/envs/ai4halth/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/ubuntu/anaconda3/envs/ai4halth/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/lin

ClassifierChain(classifier=LogisticRegression(), require_dense=[True, True])

In [26]:
predicted_labels_cc = classifier.predict(test_features)
evaluate(test_labels, predicted_labels_cc)

Accuracy: 0.08892921960072596
Hamming Loss: 0.2859735545760954
Classification Report:
               precision    recall  f1-score   support

           0       0.64      0.66      0.65       357
           1       0.30      0.29      0.29       177
           2       0.25      0.14      0.18       142
           3       0.22      0.12      0.16        98
           4       0.23      0.11      0.15        82
           5       0.10      0.05      0.06        88
           6       0.12      0.06      0.08        51

   micro avg       0.43      0.34      0.38       995
   macro avg       0.27      0.20      0.22       995
weighted avg       0.37      0.34      0.35       995
 samples avg       0.55      0.44      0.34       995

Confusion matrix:
 [[[ 61 133]
  [120 237]]

 [[253 121]
  [126  51]]

 [[348  61]
  [122  20]]

 [[411  42]
  [ 86  12]]

 [[438  31]
  [ 73   9]]

 [[428  35]
  [ 84   4]]

 [[479  21]
  [ 48   3]]]


### Label Powerset
takes correlations into account!

In [27]:
from skmultilearn.problem_transform import LabelPowerset

classifier = LabelPowerset(LogisticRegression())
classifier.fit(train_features, train_labels)

/home/ubuntu/anaconda3/envs/ai4halth/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LabelPowerset(classifier=LogisticRegression(), require_dense=[True, True])

In [28]:
predicted_labels_lp = classifier.predict(test_features)
evaluate(test_labels, predicted_labels_lp)

Accuracy: 0.10526315789473684
Hamming Loss: 0.26186155042779363
Classification Report:
               precision    recall  f1-score   support

           0       0.63      0.65      0.64       357
           1       0.36      0.23      0.28       177
           2       0.36      0.15      0.21       142
           3       0.28      0.13      0.18        98
           4       0.18      0.09      0.12        82
           5       0.11      0.01      0.02        88
           6       0.17      0.06      0.09        51

   micro avg       0.49      0.32      0.38       995
   macro avg       0.30      0.19      0.22       995
weighted avg       0.40      0.32      0.34       995
 samples avg       0.62      0.41      0.35       995

Confusion matrix:
 [[[ 61 133]
  [126 231]]

 [[302  72]
  [137  40]]

 [[372  37]
  [121  21]]

 [[419  34]
  [ 85  13]]

 [[437  32]
  [ 75   7]]

 [[455   8]
  [ 87   1]]

 [[485  15]
  [ 48   3]]]


### Multi Label KNN

In [ ]:
from skmultilearn.adapt import MLkNN
from scipy.sparse import csr_matrix, lil_matrix

mlknn = MLkNN(k=10)

x_train = lil_matrix(train_features).toarray()
y_train = lil_matrix(train_labels).toarray()
x_test = lil_matrix(test_features).toarray()
# train
mlknn.fit(x_train, y_train)
# predict
# predictions_new = classifier_new.predict(x_test)
# # accuracy
# print("Accuracy = ",accuracy_score(y_test,predictions_new))
# print("\n")
predicted_labels_mlknn = mlknn.predict(x_test)
evaluate(test_labels, predicted_labels_mlknn)